<a href="https://colab.research.google.com/github/bahron/machine_learning/blob/main/Front_View_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image Clasification - Front View Car**
**Menggunakan Dataset dari kaggle**

Link : https://www.kaggle.com/datasets/sadratmh/front-view-cars

**Mempersiapkan Dataset**

Downloan data dari kaggle

In [15]:
!wget --no-check-certificate \
https://drive.google.com/file/d/1Ajc9FobE45_vtNBY7qRd9We7lHbePo8z/view?usp=share_link \
  -O '/content/front-view-car.zip'

--2022-11-06 06:04:23--  https://drive.google.com/file/d/1Ajc9FobE45_vtNBY7qRd9We7lHbePo8z/view?usp=share_link
Resolving drive.google.com (drive.google.com)... 173.194.194.139, 173.194.194.100, 173.194.194.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.194.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/front-view-car.zip’

/content/front-view     [ <=>                ]  72.56K  --.-KB/s    in 0.003s  

2022-11-06 06:04:23 (25.0 MB/s) - ‘/content/front-view-car.zip’ saved [74303]



**Ekstrak zip file**

In [17]:
import os, zipfile

local_zip = '/content/front-view-car.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close


BadZipFile: ignored